In [1]:
from bs4 import BeautifulSoup
from bs4.element import NavigableString, Tag
import requests
import re

url = 'https://www.aonprd.com/'
href = 'ClassDisplay.aspx?ItemName=Cleric'


class ArchivesNethysScraper:
    def __init__(self, url, href):
        self.url = url
        self.href = href
        self.page = requests.get(url+href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        


    def table_rows(self):
        for b in self.soup.find_all('b')[1:]:
            yield b

            
    def get_class_info(self, table_row):
        output = {}
        tag = table_row  
        page = requests.get(url+href)
        attributes = ['Alignment', 'Hit Die', 'Starting Wealth' ]

        def fetch_info(left_tag, right_tag, format_output):
            regex = re.compile(
                f'{left_tag}(.*){right_tag}'
            )            
            return format_output(regex.findall(page.text)[0])
        format_string = lambda str: str.replace('\r', '').replace(';', '')
        for i in range(len(attributes)-1):
            l = f'<b>{attributes[i]}</b>'
            r = f'<b>{attributes[i+1]}</b>'
            output[attributes[i].lower()] = fetch_info(l, r, format_string)
        l = r
        r = '<h2 class="title">Class Skills</h2>'
        output[attributes[2].lower()] = fetch_info(l, r,format_string)
        output['description'] = fetch_info(
            '<h2 class="title">Class Skills</h2>',
            '<h2 class="title">Alternate Capstones</h2>',
            #will grab until the second part and make it all one json string
            lambda str: str.replace('\r', '')
        )
        return output            

        # if tag:
        #     output['content'] = tag.get_text()

        # return output



cleric = ArchivesNethysScraper(url, href)
i = 0

for b in cleric.table_rows():
    class_info = cleric.get_class_info(b.string)
    print(class_info)     

    i += 1

    if i == 1:
        break


{'alignment': ": A cleric's alignment must be within one step of her deity's, along either the law/chaos axis or the good/evil axis (see Additional Rules).<br />", 'hit die': ': d8.<br />', 'starting wealth': ': 4d6 x 10 gp (average 140 gp).', 'description': 'The Cleric\'s class skills are Appraise (Int), Craft (Int), Diplomacy (Cha), Heal (Wis), Knowledge (arcana) (Int), Knowledge (history) (Int), Knowledge (nobility) (Int), Knowledge (planes) (Int), Knowledge (religion) (Int), Linguistics (Int), Profession (Wis), Sense Motive (Wis), and Spellcraft (Int).<br /><br /><b>Skill Points at each Level</b>: 2 + Int modifier.<h2 class="title">Class Features</h2><table class="inner"><tr><td colspan="6">&nbsp;</td><td colspan="10"><b>Spells Per Day</b></td></tr><tr><td><b>Level</b></td><td><b>Base Attack Bonus</b></td><td><b>Fort Save</b></td><td><b>Ref Save</b></td><td><b>Will Save</b></td><td><b>Special</b></td><td><b>0</b></td><td><b>1st</b></td><td><b>2nd</b></td><td><b>3rd</b></td><td><b>4